In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Set path to line catalogue
import os
if "ARTS_DATA_PATH" not in os.environ:
    os.environ["ARTS_DATA_PATH"] = "/work/um0878/users/olemke/rtcourse/arts-xml-data/"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import typhon as ty
from pyarts import xml
from olr_module import calc_olr
from bokeh.plotting import figure, output_notebook, output_file, show

output_notebook()

In [ ]:
# Read input atmosphere
atmfield = xml.load("input/midlatitude-summer.xml")

# Scale the CO2 concentration
atmfield.scale("abs_species-CO2", 1)
# atmfield.add("T", 0)
atmfield.set("T", atmfield.get("T") + 50)

# Calculate the outgoing-longwave radiation
f, olr = calc_olr(atmfield, verbosity=0)

In [ ]:
# Plotting.
from bokeh.palettes import Magma256

wn = ty.physics.frequency2wavenumber(f) / 100  # Hz -> cm^-1

temps = [225, 250, 275, atmfield.get("T", keep_dims=False)[0]]
temp_colors = ty.plots.cmap2rgba("temperature", len(temps))

# Workaround for NaNs in OLR for first frequency
olr_nonan=olr.copy()
olr_nonan[np.isnan(olr)]=0

p = figure(width=800, aspect_ratio=3/2,
           x_range=(wn.min(), wn.max()),
           tooltips=[("x", "$x"), ("y", "$y")],
          title=rf"OLR={np.trapz(olr_nonan, f):3.2f} Wm^-2")
p.xgrid.grid_line_color = p.ygrid.grid_line_color = None
p.yaxis.axis_label="Irradiance / W m^-2 Hz^-1"
p.xaxis.axis_label="Wavenumber / cm^-1"
p.y_range.start = 0

for i, t in enumerate(temps):
    p.line(x=wn, y=np.pi * ty.physics.planck(f, t),
           line_color=Magma256[int(i/(len(temps)-1)*230) % 256],
           line_width=2,
           legend_label=f"{t:3.1f} K")

p.line(x=wn, y=olr, line_width=2, legend_label="Radiance")

show(p)